# Step 2 – Alternative

In this notebook we look at an alternative method for exporting data from MySQL to Neptune. Instead of exporting to CSV files and using the Neptune bulk loader to load the data into Neptune, we use an AWS Glue job that extracts data from MySQL and inserts it directly into Neptune.

For details on converting from a relational data model to a graph data model, see [Converting a Relational Data Model to a Graph Model](https://github.com/aws-samples/aws-dbs-refarch-graph/tree/master/src/converting-to-graph#converting-a-relational-data-model-to-a-graph-model).

<img src="https://s3.amazonaws.com/aws-neptune-customer-samples/neptune-sagemaker/images/mysql-2-neptune-02.png"/>

# Import from MySQL to Neptune

Instead of importing by way of S3 and the Neptune bulk loader, we use an AWS Glue job, `export_from_mysql_to_neptune`, to extract data from MySQL and write it directly to Neptune. The script for the job can be found [here](https://github.com/aws-samples/amazon-neptune-samples/tree/master/gremlin/glue-neptune/glue-jobs/mysql-neptune/export-from-mysql-to-neptune.py). The script uses the [glue_neptune](https://github.com/awslabs/amazon-neptune-tools/tree/master/glue-neptune) Python library.

In [ ]:
%run './glue_utils.py'
%run '../util/neptune.py'

In [ ]:
job_name = glue_resource('export_from_mysql_to_neptune')

We'll clear the database before running the import to simulate this comprising a first step in a migration to a fresh Neptune database.

In [ ]:
neptune.clear()

In [ ]:
run_job(job_name)

## Query the graph

Once again we can query the graph to review the results of the import.

In [ ]:
g = neptune.graphTraversal()

### Count the number of Product and Supplier vertices in the graph

In [ ]:
print('# Products : {}'.format(g.V().hasLabel('Product').count().next()))
print('# Suppliers: {}'.format(g.V().hasLabel('Supplier').count().next()))

### Show the products supplied by a supplier

Product vertices are connected to Supplier vertices using an outgoing SUPPLIER edge.

In [ ]:
results = (g.V().hasLabel('Supplier').limit(1).
      project('supplier', 'products').
      by('name').
      by(in_('SUPPLIER').values('name').fold()).next())

print('Supplier: {}\nProducts: {}'.format(results['supplier'], results['products']))